In [7]:
#############################
### Import libraries ###
#############################

from docplex.cp.model import *
from docplex.cp.config import get_default
import numpy as np
from Solver import *
from User import *
import time

In [8]:
#############################
### Essentials functions ###
#############################
import FunctionMain as fm

In [9]:
# ----------------- Parameters

# The file to consider
file = './file_with_optimal_val/la04.txt'

# The number of solutions that we will have in the solver during the first iteration
k = 10

# The number of layer which is fixed
nb_layers = 5

# Variable which display the solution
display_sol = False

# Variable which display the start (in a vector)
display_start = False

# Variable which display the start (in a matrix)
display_matrix = False

# Time stop criterion 
tps_max = 300

# Number of iteration stop criterion
it_max = 20

# Parameters of the first neural network
nb_hidden_layers1 = 1
nb_neurons1 = [3, 1]

# Parameters of the second neural network
nb_hidden_layers2 = 1
nb_neurons2 = [3, 1]


# The number of solutions that we will have in the solver after the first iteration
k_k = 10

In [10]:
#############################
### Main program ###
#############################

print("\n--------Main program is loading...---------")

# --------- Interaction with the solver
data = []
n, m, data, T_machine, T_duration, duration, optimalval = fm.get_data_from_file(file)

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, tasks = fm.initialize_solver(data, n, m, duration)

model, variables = solver.create_constraints(model, n, m, optimalval, T_machine)

# ------------ Solve the model
print("\nSolving the model...")
msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)

# ------------ Display the result
# fm.display_solution(msol, display_sol)
print("Model solved !")

# ---------------- Interaction with the user
print("\n--------Interaction with the user...---------")
print("\nCreating the user...")
user = User()
print("User created !")

#Get the variables of the model

list_indice, list_obj, pref, list_layers, list_equal = fm.user_preferences(msol, user, nb_layers, n, m)

# Vector of the start time of each task of each preference
starts = user.start_pref(n, m, display_start)

# Matrix of the start time of each task of each preference
matrix = user.matrix_pref(n, m, display_matrix)

# Testing the order of preferences and the differences between solutions
fm.test(n, m, user)

print("list layers : ",list_layers)

sol_layers = fm.list_list_list_start_of_tasks(n, m, list_layers)


--------Main program is loading...---------

Creating the model variables...
Model tasks_by_jobs created !

Solver initialized !

Adding precedence constraints to the solver...
Precedence constraints added !

Adding disjunctive constraints to the solver...
Disjunctive constraints added !

Solving the model...
Model solved !

--------Interaction with the user...---------

Creating the user...
User created !

Classing solutions...
Le min de list_obj est 2202144
Le max de list_obj est 2425808
Solutions classed !

Creating preferences...
Preferences created !

Testing order of preferences...
	L'ordre des préférences n'est pas cohérente

Testing differences between solutions...
	Toutes les solutions sont différentes
list layers :  [[<docplex.cp.solution.CpoSolveResult object at 0x000001A9C2DAD040>, <docplex.cp.solution.CpoSolveResult object at 0x000001A9C2DAD240>, <docplex.cp.solution.CpoSolveResult object at 0x000001A9C2DAD840>, <docplex.cp.solution.CpoSolveResult object at 0x000001A9C2DA

In [11]:
# a = 1 means that the order of the sol is 0 
# a = 0 means that the order of the sol is not 0

def activation_function(input, weights, nb_hidden_layers, nb_neurons, id_layer = 0):

    S = [model.integer_var(min=-len(input), max=len(input), name="S{}{}".format(id_layer, j)) for j in range(nb_neurons[id_layer])]
    a = [model.binary_var(name="a{}{}".format(id_layer, j)) for j in range(nb_neurons[id_layer])]

    output = []
    for j in range(nb_neurons[id_layer]):
        
        solver.add_constraint(model, S[j] == sum(input[k] * weights[id_layer][j][k] for k in range(len(input))))

        solver.add_constraint(model, a[j] == (S[j] > 0))

        if id_layer == nb_hidden_layers: 
            return a[j]
        else: 
            output.append(a[j])

    return activation_function(output, weights, nb_hidden_layers, nb_neurons, id_layer + 1)

In [12]:
####################################################################
#### NOUVELLES SOLUTIONS ET NOUVEAUX CLUSTERING A CHAQUE ITERATION
####################################################################

# --------- Call Solver constructor in Solver.py and create the variables of the model
model, solver, tasks = fm.initialize_solver(data, n, m, duration)


# ---------------- Interaction with the neural network

# Create the weights of the neural network
weights01 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons1[0])]]
weights1 = weights01 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons1[i-1])] for j in range(nb_neurons1[i])] for i in range(1, nb_hidden_layers1 + 1)]

weights02 = [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(0,j,k)) for k in range(n*m)] for j in range(nb_neurons2[0])]]
weights2 = weights02 + [[[model.integer_var(min=-1, max=1, name="w{}-{}-{}".format(i,j,k)) for k in range(nb_neurons2[i-1])] for j in range(nb_neurons2[i])] for i in range(1, nb_hidden_layers2 + 1)]


# Train the neural network

for order in range(len(sol_layers)):
    for sol in sol_layers[order]:

        output_NN1 = activation_function(sol, weights1, nb_hidden_layers1, nb_neurons1)
        output_NN2 = activation_function(sol, weights2, nb_hidden_layers2, nb_neurons2)

        if order == 0:
            solver.add_constraint(model, output_NN1 == 1)
            solver.add_constraint(model, output_NN2 == 1)
        else:
            solver.add_constraint(model, output_NN1 == 0)
            solver.add_constraint(model, output_NN2 == 0)


# Compare the result of 2 different neural networks for the same solution

tasks_starts = []
for i in range(n):
    for j in range(m):
        tasks_starts.append(model.start_of(variables[i][j]))

outputvar_NN1 = activation_function(tasks_starts, weights1, nb_hidden_layers1, nb_neurons1)
outputvar_NN2 = activation_function(tasks_starts, weights2, nb_hidden_layers2, nb_neurons2)
solver.add_constraint(model, outputvar_NN1 != outputvar_NN2)
# solver.add_constraint(model, logical_or(outputvar_NN1 == 1, outputvar_NN2 == 1))


# ------------ Solve the model
print("\nSolving the model...")

msol = model.solve()
# msol, nb_solution, runtime = solver.solve(model, k_k, n, m, variables)
# print("The number of solutions generated is :",nb_solution)

# ------------ Display the result
fm.display_solution(msol, display_sol)
print("Model solved !")


Creating the model variables...
Model tasks_by_jobs created !

Solver initialized !

Solving the model...


CpoSolverException: Solver error: Problem size limit exceeded.
CP Optimizer Community Edition solves problems with search spaces up to 2^1000.
Unrestricted version options (including academia) at https://ibm.co/2s0wqSa
